In [1]:
      # -------------------------
# Step 1: Imports & settings
# -------------------------
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, classification_report

# Model
import xgboost as xgb

# SHAP
import shap

# Output folder
OUTPUT_DIR = '/content/outputs'   # in Colab /content path
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Path to your uploaded CSV
DATA_PATH = '/content/WA_Fn-UseC_-Telco-Customer-Churn.csv'  # adjust if you put file elsewhere
TARGET = 'Churn'
RANDOM_STATE = 42

print('xgboost version:', xgb.__version__)
print('shap version:', shap.__version__)


xgboost version: 3.1.1
shap version: 0.50.0


In [2]:
# -------------------------
# Step 2: Load & quick inspect
# -------------------------
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
print('Initial shape:', df.shape)
display(df.head())
display(df.dtypes)
print('\nMissing values:\n', df.isnull().sum()[lambda s: s>0])


Initial shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


,0
customerID,object
gender,object
SeniorCitizen,int64
Partner,object
Dependents,object
tenure,int64
PhoneService,object
MultipleLines,object
InternetService,object
OnlineSecurity,object



Missing values:
 Series([], dtype: int64)


In [ ]:
# -------------------------
# Step 3: Clean & basic feature engineering
# -------------------------
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])

# Convert TotalCharges which may contain spaces
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Binary encode target
if df[TARGET].dtype == 'object':
    df[TARGET] = df[TARGET].map({'Yes':1, 'No':0})

# add a derived feature (avg charge per month)
if 'TotalCharges' in df.columns and 'tenure' in df.columns:
    df['avg_charge_per_month'] = df['TotalCharges'] / (df['tenure'].replace(0, np.nan))
    df['avg_charge_per_month'] = df['avg_charge_per_month'].fillna(df['MonthlyCharges'])

print('After cleaning shape:', df.shape)
display(df.head())


After cleaning shape: (7043, 21)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,avg_charge_per_month
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,29.850000
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,0,55.573529
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,54.075000
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,40.905556
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,75.825000


In [ ]:
# -------------------------
# Step 4: Prepare feature lists & preprocessing pipeline
# -------------------------
# Separate numeric and categorical features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET in numeric_cols:
    numeric_cols.remove(TARGET)

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print('numeric_cols:', numeric_cols)
print('cat_cols:', cat_cols)

# Preprocessing
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_transformer, numeric_cols),
    ('cat', cat_transformer, cat_cols)
], remainder='drop')

numeric_cols: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'avg_charge_per_month']
cat_cols: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [ ]:
# -------------------------
# Step 5: Train/test split & pipeline
# -------------------------
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=RANDOM_STATE)

# XGBoost classifier inside pipeline
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False,
                            eval_metric='logloss', random_state=RANDOM_STATE, n_jobs=4)

pipeline = Pipeline([('preproc', preprocessor), ('clf', xgb_clf)])


In [ ]:
# -------------------------
# Step 6: Hyperparameter tuning (RandomizedSearchCV)
# -------------------------
param_dist = {
    'clf__n_estimators': [100, 200, 400],
    'clf__max_depth': [3, 5, 7],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__subsample': [0.6, 0.8, 1.0],
    'clf__colsample_bytree': [0.6, 0.8, 1.0]
}

rs = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=12,
                        scoring='roc_auc', cv=3, verbose=2, random_state=RANDOM_STATE, n_jobs=1)

print('Starting hyperparameter search...')
rs.fit(X_train, y_train)
print('Best params:', rs.best_params_)
best_model = rs.best_estimator_
# Save model
joblib.dump(best_model, os.path.join(OUTPUT_DIR, 'best_model.joblib'))
print('Model saved to', os.path.join(OUTPUT_DIR, 'best_model.joblib'))


Starting hyperparameter search...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=7, clf__n_estimators=400, clf__subsample=0.6; total time=   0.9s
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=7, clf__n_estimators=400, clf__subsample=0.6; total time=   0.9s
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=7, clf__n_estimators=400, clf__subsample=0.6; total time=   0.9s
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=400, clf__subsample=0.6; total time=   0.4s
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=400, clf__subsample=0.6; total time=   0.4s
[CV] END clf__colsample_bytree=0.6, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=400, clf__subsample=0.6; total time=   0.4s
[CV] END clf__colsample_bytree=0.8, clf__learning_rate=0.1,

In [ ]:
# -------------------------
# Step 7: Evaluation on test set
# -------------------------
# Predictions
X_test_trans = best_model.named_steps['preproc'].transform(X_test)  # for later shap
y_proba = best_model.predict_proba(X_test)[:,1]
y_pred = (y_proba >= 0.5).astype(int)

metrics = {
    'auc': roc_auc_score(y_test, y_proba),
    'f1': f1_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred)
}
print('Test metrics:', metrics)
print('Classification report:\n', classification_report(y_test, y_pred))

# Save metrics
with open(os.path.join(OUTPUT_DIR, 'metrics.txt'), 'w') as f:
    f.write('Test metrics:\n')
    f.write(str(metrics) + '\n\n')
    f.write(classification_report(y_test, y_pred))


Test metrics: {'auc': np.float64(0.8478738794595572), 'f1': 0.5696594427244582, 'recall': 0.4919786096256685, 'precision': 0.6764705882352942}
Classification report:
               precision    recall  f1-score   support

           0       0.83      0.91      0.87      1035
           1       0.68      0.49      0.57       374

    accuracy                           0.80      1409
   macro avg       0.75      0.70      0.72      1409
weighted avg       0.79      0.80      0.79      1409



In [ ]:
# -------------------------
# Step 8: Prepare feature names after preprocessing
# -------------------------
# We need the transformed feature names for SHAP plotting
preproc = best_model.named_steps['preproc']
# numeric names are numeric_cols (in listed order)
num_names = numeric_cols

# get onehot encoder output names
ohe = None
cat_input_cols = []
for name, transformer, cols in preproc.transformers_:
    if name == 'cat':
        # transformer is the pipeline for cat
        ohe = transformer.named_steps['onehot']
        cat_input_cols = cols

if ohe is not None:
    try:
        ohe_cols = list(ohe.get_feature_names_out(cat_input_cols))
    except Exception:
        # older sklearn compatibility
        ohe_cols = list(ohe.get_feature_names(cat_input_cols))
else:
    ohe_cols = []

feature_names = list(num_names) + ohe_cols
print('Number of features after preprocessing:', len(feature_names))
print('Sample feature names:', feature_names[:30])


Number of features after preprocessing: 46
Sample feature names: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'avg_charge_per_month', 'gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No', 'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes', 'MultipleLines_No', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_No internet service']


In [ ]:
# -------------------------
# Step 9: SHAP explainability (TreeExplainer)
# -------------------------
# We'll use TreeExplainer which is fast for tree models.
# For large models/data you can use a sample to compute SHAP values.

# Use a sample if you want speed:
sample_size = min(2000, X_test.shape[0])
X_shap_sample = X_test.sample(sample_size, random_state=RANDOM_STATE)

# Transform sample
X_shap_trans = preproc.transform(X_shap_sample)

# Create explainer and compute shap values
try:
    # Older/standard approach for tree models
    explainer = shap.TreeExplainer(best_model.named_steps['clf'])
    shap_values = explainer.shap_values(X_shap_trans)
    # shap_values shape: (n_samples, n_features) for binary
    print('Computed SHAP values with TreeExplainer (classic).')
except Exception as e:
    print('TreeExplainer failed, trying shap.Explainer fallback:', e)
    explainer = shap.Explainer(best_model.named_steps['clf'], masker=shap.maskers.Independent(X_shap_trans))
    shap_vals_obj = explainer(X_shap_trans)   # returns a ShapValues object
    shap_values = shap_vals_obj.values
    print('Computed SHAP values with shap.Explainer fallback.')

# If shap_values is a list for some versions, handle it
if isinstance(shap_values, list):
    # choose class 1 (positive class) SHAP values
    shap_values = shap_values[1]

print('shap_values shape:', np.array(shap_values).shape)


Computed SHAP values with TreeExplainer (classic).
shap_values shape: (1409, 46)


In [ ]:
# -------------------------
# Step 10: Global SHAP plots (summary & bar)
# -------------------------
# Summary plot (dot)
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_shap_trans, feature_names=feature_names, show=False)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'shap_summary.png'), dpi=150)
plt.close()
print('Saved shap_summary.png')

# Bar plot (mean |shap|)
plt.figure(figsize=(8,6))
shap.summary_plot(shap_values, X_shap_trans, feature_names=feature_names, plot_type='bar', show=False)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'shap_bar.png'), dpi=150)
plt.close()
print('Saved shap_bar.png')


Saved shap_summary.png
Saved shap_bar.png


In [ ]:
# -------------------------
# Step 11: Top positive & negative drivers
# -------------------------
# Compute mean SHAP per feature across the sample
mean_shap = np.mean(shap_values, axis=0)            # positive means pushes toward churn
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)

feat_df = pd.DataFrame({
    'feature': feature_names,
    'mean_shap': mean_shap,
    'mean_abs_shap': mean_abs_shap
}).sort_values('mean_abs_shap', ascending=False)

top_pos = feat_df.sort_values('mean_shap', ascending=False).head(8)   # top drivers that increase churn
top_neg = feat_df.sort_values('mean_shap', ascending=True).head(8)    # top drivers that reduce churn

print('Top features that INCREASE churn (positive mean SHAP):')
display(top_pos.head(10))
print('\nTop features that DECREASE churn (negative mean SHAP):')
display(top_neg.head(10))

feat_df.to_csv(os.path.join(OUTPUT_DIR, 'shap_feature_stats.csv'), index=False)
print('Saved shap_feature_stats.csv')


Top features that INCREASE churn (positive mean SHAP):


,feature,mean_shap,mean_abs_shap
34,StreamingMovies_No,0.001387,0.005308
27,DeviceProtection_Yes,0.000524,0.001502
31,StreamingTV_No,0.000410,0.008914
24,OnlineBackup_Yes,0.000323,0.004879
23,OnlineBackup_No internet service,0.000262,0.002398
5,gender_Female,0.000258,0.003941
42,PaymentMethod_Bank transfer (automatic),0.000098,0.001472
29,TechSupport_No internet service,0.000000,0.000000



Top features that DECREASE churn (negative mean SHAP):


,feature,mean_shap,mean_abs_shap
37,Contract_Month-to-month,-0.082576,0.548592
1,tenure,-0.077524,0.312626
17,InternetService_Fiber optic,-0.039260,0.220671
39,Contract_Two year,-0.038161,0.149420
28,TechSupport_No,-0.036267,0.171965
19,OnlineSecurity_No,-0.032805,0.208572
2,MonthlyCharges,-0.031427,0.123942
44,PaymentMethod_Electronic check,-0.023538,0.145591


Saved shap_feature_stats.csv


In [ ]:
# -------------------------
# Step 12: Dependence plots for top features
# -------------------------
top_feats_by_abs = feat_df.sort_values('mean_abs_shap', ascending=False)['feature'].tolist()[:5]
for feat in top_feats_by_abs:
    try:
        plt.figure(figsize=(8,6))
        shap.dependence_plot(feat, shap_values, X_shap_trans, feature_names=feature_names, show=False)
        plt.tight_layout()
        safe = feat.replace(' ','_').replace(':','_')[:120]
        plt.savefig(os.path.join(OUTPUT_DIR, f'shap_dependence_{safe}.png'), dpi=150)
        plt.close()
        print('Saved dependence for', feat)
    except Exception as e:
        print('Dependence plot failed for', feat, e)


Saved dependence for Contract_Month-to-month
Saved dependence for tenure
Saved dependence for InternetService_Fiber optic
Saved dependence for OnlineSecurity_No
Saved dependence for TechSupport_No


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [ ]:
# -------------------------
# Step 13: Local explanations (save force plots for 3 representative customers)
# -------------------------
# We'll compute predicted probabilities on the full dataset to choose examples
X_trans_full = preproc.transform(X)
probs_full = best_model.named_steps['clf'].predict_proba(X_trans_full)[:,1]

df_examples = X.copy()
df_examples['_pred_proba'] = probs_full
# select indices (high, low, borderline)
high_idx = df_examples['_pred_proba'].idxmax()
low_idx = df_examples['_pred_proba'].idxmin()
border_idx = (df_examples['_pred_proba'] - 0.5).abs().idxmin()
selected_idx = [high_idx, low_idx, border_idx]
print('Selected indices:', selected_idx)

# compute shap on a small set that includes these three (to ensure they are available)
# transform selected rows
rows = X.loc[selected_idx]
rows_trans = preproc.transform(rows)

# recompute shap values for those rows using the explainer object
try:
    # If we used TreeExplainer earlier, it can compute shap_values for new data
    shap_vals_selected = explainer.shap_values(rows_trans)
    if isinstance(shap_vals_selected, list):
        shap_vals_selected = shap_vals_selected[1]
except Exception as e:
    # Use explainer(rows_trans) if using shap.Explainer fallback
    shap_obj = explainer(rows_trans)
    shap_vals_selected = shap_obj.values

# Save interactive HTML force plots and png fallback
for i, idx in enumerate(selected_idx):
    row = rows.iloc[[i]]
    row_trans = rows_trans[i:i+1]
    vals = shap_vals_selected[i]
    try:
        # interactive html (recommended)
        force = shap.force_plot(explainer.expected_value, vals, row_trans, feature_names=feature_names, matplotlib=False)
        html_path = os.path.join(OUTPUT_DIR, f'force_{idx}.html')
        shap.save_html(html_path, force)
        print('Saved interactive force plot (html) ->', html_path)
    except Exception as e:
        # matplotlib fallback PNG
        try:
            plt.figure(figsize=(8,3))
            shap.force_plot(explainer.expected_value, vals, row_trans, feature_names=feature_names, matplotlib=True, show=False)
            png_path = os.path.join(OUTPUT_DIR, f'force_{idx}.png')
            plt.tight_layout()
            plt.savefig(png_path, dpi=150)
            plt.close()
            print('Saved force plot (png) ->', png_path)
        except Exception as e2:
            print('Failed to save force plot for', idx, e, e2)

# Save the three example rows with predictions for your report
examples_df = df.copy().loc[selected_idx]
examples_df['_pred_proba'] = probs_full[selected_idx]
examples_df.to_csv(os.path.join(OUTPUT_DIR, 'three_example_customers.csv'), index=True)
print('Saved three_example_customers.csv')


Selected indices: [2208, 2524, 2761]
Saved interactive force plot (html) -> /content/outputs/force_2208.html
Saved interactive force plot (html) -> /content/outputs/force_2524.html
Saved interactive force plot (html) -> /content/outputs/force_2761.html
Saved three_example_customers.csv


In [ ]:
# -------------------------
# Step 14: Final notes & quick report template
# -------------------------
print('All outputs saved in', OUTPUT_DIR)
print('Files:', os.listdir(OUTPUT_DIR))

# Quick report template — fill with results above
report_text = f"""
MODEL & METRICS
- Model: XGBoost (tuned)
- Test AUC: {metrics['auc']:.4f}
- Test F1: {metrics['f1']:.4f}
- Test Recall: {metrics['recall']:.4f}
- Test Precision: {metrics['precision']:.4f}

TOP GLOBAL SHAP INSIGHTS (examples)
- Top positive drivers (increase churn): {', '.join(top_pos['feature'].head(5).tolist())}
- Top negative drivers (reduce churn): {', '.join(top_neg['feature'].head(5).tolist())}

LOCAL EXAMPLES:
- High-risk customer index: {high_idx}, predicted_prob = {df_examples.loc[high_idx,'_pred_proba']:.3f}
- Low-risk customer index: {low_idx}, predicted_prob = {df_examples.loc[low_idx,'_pred_proba']:.3f}
- Borderline customer index: {border_idx}, predicted_prob = {df_examples.loc[border_idx,'_pred_proba']:.3f}

RECOMMENDATIONS (example templates based on SHAP drivers — customize before submitting)
1) If 'tenure' or low usage increases churn: implement early re-engagement email + 30-day free trial add-on for customers with tenure < X months.
2) If 'Contract=Month-to-month' increases churn: target month-to-month customers approaching month 11 with one-time discounted 12-month offers.
3) If 'TechSupport' or 'InternetService' issues increase churn: auto-escalate customers with support tickets > Y to retention team with tailored offers.

Files saved: shap_summary.png, shap_bar.png, shap_dependence_*.png, force_*.html, best_model.joblib, metrics.txt, shap_feature_stats.csv, three_example_customers.csv.
"""
with open(os.path.join(OUTPUT_DIR, 'quick_report.txt'), 'w') as f:
    f.write(report_text)

print('Wrote quick_report.txt with a template you can copy into your submission.')
print(report_text)


All outputs saved in /content/outputs
Files: ['shap_dependence_TechSupport_No.png', 'force_2208.html', 'shap_dependence_tenure.png', 'shap_dependence_OnlineSecurity_No.png', 'shap_dependence_InternetService_Fiber_optic.png', 'shap_feature_stats.csv', 'three_example_customers.csv', 'shap_bar.png', 'force_2761.html', 'shap_summary.png', 'shap_dependence_Contract_Month-to-month.png', 'force_2524.html', 'metrics.txt', 'best_model.joblib']
Wrote quick_report.txt with a template you can copy into your submission.

MODEL & METRICS
- Model: XGBoost (tuned)
- Test AUC: 0.8479
- Test F1: 0.5697
- Test Recall: 0.4920
- Test Precision: 0.6765

TOP GLOBAL SHAP INSIGHTS (examples)
- Top positive drivers (increase churn): StreamingMovies_No, DeviceProtection_Yes, StreamingTV_No, OnlineBackup_Yes, OnlineBackup_No internet service
- Top negative drivers (reduce churn): Contract_Month-to-month, tenure, InternetService_Fiber optic, Contract_Two year, TechSupport_No

LOCAL EXAMPLES:
- High-risk customer i